In [1]:
# libraries to import
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from ast import literal_eval
from typing import Dict
import os
print(os.getcwd())
os.chdir(r"C:\Users\LMENENDEZ\GitHub\MultiModX")
print(os.getcwd())
pd.set_option('display.max_columns', None)

c:\Users\LMENENDEZ\GitHub\MultiModX\notebooks
C:\Users\LMENENDEZ\GitHub\MultiModX


In [ ]:
%load_ext autoreload

In [3]:
%autoreload
from strategic_evaluator.logit_model import *

In [4]:
# Trips during the week 22/09/2022 28/09/2022 (thursday to thursday)
# the day of study selected was Friday to put the air layer under pressure
all_trips = pd.read_csv(
    r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP4 Performance Assessment Solution\Demand data\Matrices MITMA\with_archetypes\20220922_28_trip_matrix_arc_pt_processed.csv.gz",
    compression="gzip",
    sep="|"
)

In [5]:
#Here there is no trip id but later trip id appears all the time
trips = all_trips[all_trips["date"] == 20220923].reset_index(drop=True).rename(columns={"origin_nut": "origin", "destination_nut": "destination"})
trips.head()

,date,trip_period,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,distance,route_distance,duration,mode,service,legs,trip_vehicle_type,nationality,home_census,home_zone,overnight_census,income,age,sex,vehicle_type,short_professional_driver,trips,trips_km,sample_trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced
0,20220923,P00,01002,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,NF,O,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*01002*01002*None*train_13121*00-01*road*No...,other,ES,2_48,4801303,2_48,I01_[10000-15000),A01_[25-45),male,passenger,False,4.135,139.516,1.0,0.000000,0.000000,2.067500,2.067500,0.000000,0.000000,3,road-train-road,train_13121-train_13200,train_13121,train_13200,national,2,1,0,train_13121-train_13200,train_13106,train_13200
1,20220923,P00,01009_AM,ES211,Álava,31010_AM,ES220,Navarra,NaN,NaN,O,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,ES,2_31,31010_AM,2_31,I01_[10000-15000),A01_[25-45),male,passenger,False,5.191,69.312,1.0,3.707857,0.000000,0.000000,0.494381,0.000000,0.988762,3,road-train-road,train_11213-train_11300,train_11213,train_11300,national,2,1,0,train_11213-train_11300,train_13106,train_11300
2,20220923,P00,01009_AM,ES211,Álava,abroad_208,abroad,abroad,NaN,ground_Fra_08,NF,NF,abroad,D05_[50000-inf),01-02,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,FR,NaN,NaN,NaN,NaN,NaN,NaN,passenger,False,1.599,309.588,1.0,0.888333,0.399750,0.142133,0.071067,0.062183,0.035533,3,road-train-road,train_11213-train_11600,train_11213,train_11600,international_D,2,1,0,train_11213-train_11600,train_11208,train_11511
3,20220923,P00,01036,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,H,O,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01036*01036*None*train_13106*00-01*road*No...,other,ES,2_01,01036,2_48,I02_[15000-inf),A02_[45-65),female,passenger,False,6.236,139.477,1.0,3.118000,0.000000,0.779500,1.559000,0.779500,0.000000,3,road-train-road,train_13106-train_13200,train_13106,train_13200,national,2,1,0,train_13106-train_13200,train_13106,train_13200
4,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,O,NF,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_01,0105904,2_01,I02_[15000-inf),A02_[45-65),male,passenger,False,4.215,155.813,1.0,3.417568,0.113919,0.227838,0.227838,0.227838,0.000000,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200


In [6]:
#associates each airport to the corresponding new NUTS for the deprecated NUTS
airports_to_NUTS={"airport_LPA":("ES705","Gran Canaria"),
                 "airport_FUE":("ES704","Fuerteventura"),
                 "airport_ACE":("ES708","Lanzarote"),
                 "airport_TFS":("ES709","Tenerife"),
                 "airport_TFN":("ES709","Tenerife"),
                 "airport_SPC":("ES707","La Palma"),
                 "airport_VDE":("ES703","El Hierro"),
                 "airport_PMI":("ES532","Mallorca"),
                 "airport_IBZ":("ES531","Eivissa i Formentera"),
                 "airport_MAH":("ES533","Menorca")
                 }

In [7]:
def format_trips(trips: pd.DataFrame, airports_to_NUTS: dict):
    # Filter trips (only those without "abroad" as origin or destination)
    trips = trips[~((trips["origin"] == "abroad") | (trips["destination"] == "abroad"))].copy()

    # Modify 'mode_tp' column: replacing modes with specific terminology
    trips.loc[:, "mode_tp"] = (
        trips["mode_sequence"]
        .str.replace("bus", "road")  # replace bus to road (some people can reach infrastructure by bus)
        .str.replace("plane", "air")  # use nomenclature of the offer data
        .str.replace("train", "rail")
    )

    # Remove "road" from the 'mode_tp' column
    trips.loc[:, "mode_tp"] = trips["mode_tp"].apply(
        lambda row: [mode for mode in row.split("-") if mode != "road"]
    )  # remove "road" (it will be considered like access time)

    # Only consider trips that do not contain "ship"
    trips = trips[~trips["mode_tp"].apply(lambda x: "ship" in x)]

    # Change aggregated island NUTS to dis-aggregated NUTS
    for key in airports_to_NUTS.keys():
        trips.loc[trips["start_node"] == key, ["origin", "origin_name"]] = [
            airports_to_NUTS[key][0],
            airports_to_NUTS[key][1]
        ]  # change start node
        trips.loc[trips["end_node"] == key, ["destination", "destination_name"]] = [
            airports_to_NUTS[key][0],
            airports_to_NUTS[key][1]
        ]  # change destination node

    return trips

In [8]:
trips=format_trips(trips,airports_to_NUTS)

In [18]:
trips.head()

,date,trip_period,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,distance,route_distance,duration,mode,service,legs,trip_vehicle_type,nationality,home_census,home_zone,overnight_census,income,age,sex,vehicle_type,short_professional_driver,trips,trips_km,sample_trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp
0,20220923,P00,01002,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,NF,O,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*01002*01002*None*train_13121*00-01*road*No...,other,ES,2_48,4801303,2_48,I01_[10000-15000),A01_[25-45),male,passenger,False,4.135,139.516,1.0,0.000000,0.000000,2.067500,2.067500,0.000000,0.000000,3,road-train-road,train_13121-train_13200,train_13121,train_13200,national,2,1,0,train_13121-train_13200,train_13106,train_13200,[rail]
1,20220923,P00,01009_AM,ES211,Álava,31010_AM,ES220,Navarra,NaN,NaN,O,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,ES,2_31,31010_AM,2_31,I01_[10000-15000),A01_[25-45),male,passenger,False,5.191,69.312,1.0,3.707857,0.000000,0.000000,0.494381,0.000000,0.988762,3,road-train-road,train_11213-train_11300,train_11213,train_11300,national,2,1,0,train_11213-train_11300,train_13106,train_11300,[rail]
3,20220923,P00,01036,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,H,O,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01036*01036*None*train_13106*00-01*road*No...,other,ES,2_01,01036,2_48,I02_[15000-inf),A02_[45-65),female,passenger,False,6.236,139.477,1.0,3.118000,0.000000,0.779500,1.559000,0.779500,0.000000,3,road-train-road,train_13106-train_13200,train_13106,train_13200,national,2,1,0,train_13106-train_13200,train_13106,train_13200,[rail]
4,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,O,NF,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_01,0105904,2_01,I02_[15000-inf),A02_[45-65),male,passenger,False,4.215,155.813,1.0,3.417568,0.113919,0.227838,0.227838,0.227838,0.000000,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200,[rail]
5,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,W,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_09,09219,2_09,I01_[10000-15000),A02_[45-65),male,passenger,False,3.574,133.590,1.0,2.757086,0.000000,0.204229,0.102114,0.000000,0.510571,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200,[rail]


In [32]:
trips_not_commuting=trips[~(((trips["origin_purpose"]=="H")&(trips["destination_purpose"]=="W"))|((trips["origin_purpose"]=="W")&(trips["destination_purpose"]=="H")))]

In [33]:
trips_not_commuting.head()

,date,trip_period,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,distance,route_distance,duration,mode,service,legs,trip_vehicle_type,nationality,home_census,home_zone,overnight_census,income,age,sex,vehicle_type,short_professional_driver,trips,trips_km,sample_trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,duration_min,duration_max
0,20220923,P00,01002,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,NF,O,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*01002*01002*None*train_13121*00-01*road*No...,other,ES,2_48,4801303,2_48,I01_[10000-15000),A01_[25-45),male,passenger,False,4.135,139.516,1.0,0.000000,0.000000,2.067500,2.067500,0.000000,0.000000,3,road-train-road,train_13121-train_13200,train_13121,train_13200,national,2,1,0,train_13121-train_13200,train_13106,train_13200,[rail],60.0,120.0
1,20220923,P00,01009_AM,ES211,Álava,31010_AM,ES220,Navarra,NaN,NaN,O,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,ES,2_31,31010_AM,2_31,I01_[10000-15000),A01_[25-45),male,passenger,False,5.191,69.312,1.0,3.707857,0.000000,0.000000,0.494381,0.000000,0.988762,3,road-train-road,train_11213-train_11300,train_11213,train_11300,national,2,1,0,train_11213-train_11300,train_13106,train_11300,[rail],0.0,60.0
3,20220923,P00,01036,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,H,O,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01036*01036*None*train_13106*00-01*road*No...,other,ES,2_01,01036,2_48,I02_[15000-inf),A02_[45-65),female,passenger,False,6.236,139.477,1.0,3.118000,0.000000,0.779500,1.559000,0.779500,0.000000,3,road-train-road,train_13106-train_13200,train_13106,train_13200,national,2,1,0,train_13106-train_13200,train_13106,train_13200,[rail],0.0,60.0
4,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,O,NF,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_01,0105904,2_01,I02_[15000-inf),A02_[45-65),male,passenger,False,4.215,155.813,1.0,3.417568,0.113919,0.227838,0.227838,0.227838,0.000000,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200,[rail],60.0,120.0
6,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,W,NF,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_01,0105904,2_01,I02_[15000-inf),A01_[25-45),male,passenger,False,4.660,176.359,1.0,4.349333,0.000000,0.077667,0.000000,0.233000,0.000000,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200,[rail],0.0,60.0


In [23]:
trips["trips"].sum()

np.float64(521582.34700000007)

In [34]:
trips_not_commuting["trips"].sum()

np.float64(501635.66599999997)

The majority of people travel in between NUTS for non-commuting reasons. Let us see if the current assignation is still valid

In [25]:
# Function to parse the trip_duration and calculate min and max durations
def parse_duration(row):
    # Remove the square brackets and split the string by '-'
    duration_str = row['duration'][:]  # Removing '[' and ']'
    min_val, max_val = duration_str.split('-')

    # Convert the values to minutes, handling 'inf'
    if min_val == 'inf':
        min_duration = float('inf')
    else:
        min_duration = int(min_val) * 60  # Convert to minutes

    if max_val == 'inf':
        max_duration = float('inf')
    else:
        max_duration = int(max_val) * 60  # Convert to minutes

    return pd.Series([min_duration, max_duration])


In [35]:
trips[['duration_min', 'duration_max']] = trips.apply(parse_duration, axis=1)
trips_not_commuting=trips_not_commuting.copy()
trips_not_commuting[['duration_min', 'duration_max']] = trips_not_commuting.apply(parse_duration, axis=1)

In [36]:
# location of "ALL" train stops given by UiC
# However this list is still incomplete
stops_loc=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\infrastructure\rail_info\stops.txt").astype(str) # everything is a string here to match other formatting
stops_loc["stop_id"] = stops_loc["stop_id"].apply(lambda x: "00" + x) #to make they start with 00

In [37]:
#finds weird stations
def find_weird_stations(node_sequence,stops_loc):
    weird_stations=[]
    nodes = node_sequence.split("-")
    for node in nodes:
        if node.startswith("train_"):
            station_id=node.split("_")[1]
            if not station_id.isdigit():
                weird_stations.append(station_id)
            else:
                station_id_modified_1 = f"0071{int(station_id):05d}"
                station_id_modified_2 = f"0087{int(station_id):05d}"
                station_id_modified_3 = f"0094{int(station_id):05d}"
                if any(station_id in stops_loc["stop_id"].values for station_id in [station_id_modified_1, station_id_modified_2, station_id_modified_3]):
                    pass
                else:
                    weird_stations.append(station_id)
    return weird_stations

In [38]:
trips["weird_stations"] = trips["node_sequence_reduced"].apply(
    lambda x: find_weird_stations(x, stops_loc))
trips_not_commuting["weird_stations"] = trips_not_commuting["node_sequence_reduced"].apply(
    lambda x: find_weird_stations(x, stops_loc))

In [39]:
# Assuming the "weird_stations" column contains lists
# Flatten the lists into one combined list
all_weird_stations = trips["weird_stations"].explode().dropna()

# Extract unique values
unique_weird_stations = all_weird_stations.unique()

# Convert back to a list if needed
unique_weird_stations = list(unique_weird_stations)
# converts this list into Moba format
unique_weird_stations_modified=[ "train_"+ station for station in unique_weird_stations] 

In [ ]:
# read files with information about ALL stations considered by MobA
MobA_stations_coord=gpd.read_file(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\oferta_transporte\train_stations\train_stations.shp")

In [41]:
# identifies all the un-localisable stations
nowhere_stations=set(unique_weird_stations_modified)-set(MobA_stations_coord["ID"])
print(f"there are {len(nowhere_stations)} stations that are not in the data provided by MobA but appear in the trips dataframe")

there are 197 stations that are not in the data provided by MobA but appear in the trips dataframe


In [42]:
trips = trips[~trips["node_sequence_reduced"].apply(lambda x: any(station in x for station in nowhere_stations))]
trips_not_commuting = trips_not_commuting[~trips_not_commuting["node_sequence_reduced"].apply(lambda x: any(station in x for station in nowhere_stations))]

In [43]:
# coordinates, geometry and other properties of all NUTS (in Europe?)
NUTS_coord=gpd.read_file(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP4 Performance Assessment Solution\Demand data\nuts3_2003_geom_10.gpkg")

In [44]:
# align coordinate reference system (crs)
NUTS_coord = NUTS_coord.to_crs(MobA_stations_coord.crs)

# Perform a spatial join to find which NUTS region each station belongs to
spatial_join = gpd.sjoin(MobA_stations_coord, NUTS_coord, how="left", predicate="within")

# Construct the dictionary
station_to_NUTS = dict(zip(spatial_join["ID"], spatial_join["geocode"]))

In [45]:
# list of stations considered in MMX
train_stations_considered=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\infrastructure\rail_info\rail_stations_considered_GTFS_2022.csv").astype(str)
train_stations_considered["stop_id"] = train_stations_considered["stop_id"].apply(lambda x: "00" + x) #to make they start with 00

In [46]:
#format train_station considered
def format_train_stations_considered(train_stations_considered: pd.DataFrame,station_to_nuts: Dict):
    train_stations_considered["modified_id"]=train_stations_considered["stop_id"].apply(lambda x : "train_" + str(x)[4:])
    train_stations_considered['NUTS'] = train_stations_considered['modified_id'].map(station_to_nuts)
    return train_stations_considered

In [69]:
train_stations_considered=format_train_stations_considered(train_stations_considered,station_to_NUTS)

In [70]:
# build second dictionary
NUTS_to_MMX_train_station={}
for nuts, station in zip(train_stations_considered["NUTS"], train_stations_considered["stop_id"]):
    if nuts in NUTS_to_MMX_train_station:
        NUTS_to_MMX_train_station[nuts].append(station)
    else:
        NUTS_to_MMX_train_station[nuts]=[station]

In [71]:
station_to_station_MMX = {}

for station, nuts_code in station_to_NUTS.items():
    # Get the value from nuts_to_train_stations_considered using the nuts_code
    if nuts_code in NUTS_to_MMX_train_station:
        station_to_station_MMX[station] = NUTS_to_MMX_train_station[nuts_code]

In [72]:
# construct a dictionary with the equivalence between IATA and ICAO codes
airports=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\infrastructure\airports_info\IATA_ICAO_Airport_codes_v1.3.csv")
airports["ICAO"] = airports["ICAO"].fillna(airports["IATA"])
IATA_to_ICAO = airports.set_index("IATA")["ICAO"].to_dict()

In [75]:
# list of all MMX stations
train_station_MMX=train_stations_considered["stop_id"].tolist()

In [73]:
# to build a node sequence that consisting in MMX nodes
def process_node_sequence_MMX(trips: pd.DataFrame, train_stations_MMX: list, IATA_to_ICAO: dict):
   # finds node sequence in MMX (first attempt)

    def process_row(row):
        # checks whether the corresponding column is a string
        node_sequence = row['node_sequence_reduced']
        if not isinstance(node_sequence, str):
            return np.nan

        result = []
        for split in node_sequence.split("-"):
            if split.startswith("train"):
                split=split.replace("train_", "")
                if split.isalpha():
                    return np.nan
                elif f"0071{int(split):05d}" in train_stations_MMX:
                    result.append(f"0071{int(split):05d}")
                else:
                    return np.nan
            elif split.startswith("airport_"):
                iata_code = split.replace("airport_", "")
                icao_code = IATA_to_ICAO.get(iata_code)
                if icao_code:
                    result.append(icao_code)
                else:
                    return np.nan
        return result

    trips = trips.copy()
    # Apply the processing function to each row in the dataframe
    trips.loc[:,'node_sequence_MMX'] = trips.apply(process_row, axis=1)

    return trips


In [76]:
trips_not_commuting=process_node_sequence_MMX(trips_not_commuting,train_station_MMX,IATA_to_ICAO)

In [77]:
trips_with_no_path = trips_not_commuting[trips_not_commuting["node_sequence_MMX"].isna()]["trips"].sum()
print(f"There are {trips_with_no_path:.2f} trips with no path, i.e, {trips_with_no_path/trips_not_commuting["trips"].sum()*100:.2f}% of the total")

There are 213920.15 trips with no path, i.e, 43.36% of the total


This is marginally better than considering the commuting trips

In [78]:
possible_itineraries_clustered_pareto_1_connection=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\processed_baseline\paths_itineraries\possible_itineraries_clustered_pareto_0.csv")
possible_itineraries_clustered_pareto_2_connections=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\processed_baseline_2_connections\paths_itineraries\possible_itineraries_clustered_pareto_0.csv")

In [79]:
possible_itineraries_1_connection=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\processed_baseline\paths_itineraries\possible_itineraries_0.csv")
possible_itineraries_2_connections=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\processed_baseline_2_connections\paths_itineraries\possible_itineraries_0.csv")

In [80]:
def format_itineraries(itineraries: pd.DataFrame):
    # Create a copy of the filtered DataFrame to avoid potential views
    itineraries = itineraries[itineraries["nservices"] != 0].copy() # removes trips that are only access and egress
    
    # Select the columns to iterate through
    mode_columns = [col for col in itineraries.columns if col.startswith("mode_")]
    
    # Safely assign the new 'mode_tp' column using .loc
    if mode_columns!=[]:
        itineraries.loc[:, "mode_tp"] = itineraries.apply(
            lambda row: [row[col] for col in mode_columns if str(row[col]) != "nan"],
            axis=1
        )
    return itineraries

In [81]:
possible_itineraries_clustered_pareto_2_connections=format_itineraries(possible_itineraries_clustered_pareto_2_connections)
possible_itineraries_clustered_pareto_1_connection=format_itineraries(possible_itineraries_clustered_pareto_1_connection)
possible_itineraries_2_connections=format_itineraries(possible_itineraries_2_connections)
possible_itineraries_1_connection=format_itineraries(possible_itineraries_1_connection)

In [82]:
trips_not_commuting["node_sequence_MMX"]=trips_not_commuting["node_sequence_MMX"].astype(str)
trips_not_commuting=trips_not_commuting.rename(columns={"node_sequence_MMX":"path"})

In [83]:
trips_not_commuting

,date,trip_period,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,distance,route_distance,duration,mode,service,legs,trip_vehicle_type,nationality,home_census,home_zone,overnight_census,income,age,sex,vehicle_type,short_professional_driver,trips,trips_km,sample_trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,duration_min,duration_max,weird_stations,path
0,20220923,P00,01002,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,NF,O,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*01002*01002*None*train_13121*00-01*road*No...,other,ES,2_48,4801303,2_48,I01_[10000-15000),A01_[25-45),male,passenger,False,4.135,139.516,1.0,0.000000,0.000000,2.067500,2.067500,0.000000,0.000000,3,road-train-road,train_13121-train_13200,train_13121,train_13200,national,2,1,0,train_13121-train_13200,train_13106,train_13200,[rail],60.0,120.0,[13121],nan
1,20220923,P00,01009_AM,ES211,Álava,31010_AM,ES220,Navarra,NaN,NaN,O,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,ES,2_31,31010_AM,2_31,I01_[10000-15000),A01_[25-45),male,passenger,False,5.191,69.312,1.0,3.707857,0.000000,0.000000,0.494381,0.000000,0.988762,3,road-train-road,train_11213-train_11300,train_11213,train_11300,national,2,1,0,train_11213-train_11300,train_13106,train_11300,[rail],0.0,60.0,[],nan
3,20220923,P00,01036,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,H,O,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01036*01036*None*train_13106*00-01*road*No...,other,ES,2_01,01036,2_48,I02_[15000-inf),A02_[45-65),female,passenger,False,6.236,139.477,1.0,3.118000,0.000000,0.779500,1.559000,0.779500,0.000000,3,road-train-road,train_13106-train_13200,train_13106,train_13200,national,2,1,0,train_13106-train_13200,train_13106,train_13200,[rail],0.0,60.0,[],"['007113106', '007113200']"
4,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,O,NF,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_01,0105904,2_01,I02_[15000-inf),A02_[45-65),male,passenger,False,4.215,155.813,1.0,3.417568,0.113919,0.227838,0.227838,0.227838,0.000000,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200,[rail],60.0,120.0,[],"['007111208', '007111200']"
6,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,W,NF,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_01,0105904,2_01,I02_[15000-inf),A01_[25-45),male,passenger,False,4.660,176.359,1.0,4.349333,0.000000,0.077667,0.000000,0.233000,0.000000,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200,[rail],0.0,60.0,[],"['007111208', '007111200']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227596,20220923,P23,5029709,ES243,Zaragoza,44056_AM,ES242,Teruel,NaN,NaN,NF,NF,D05_[50000-inf),D05_[50000-inf),01-02,train,conv_unknown,P23*5029709*5029703*None*train_70806*00-01*roa...,other,ES,2_22,22048,2_22,I01_[10000-15000),A02_[45-65),female,passenger,False,3.468,630.518,1.0,2.813660,0.007270,0.029082,0.152679,0.247195,0.218113,3,road-train-road,train_70806-train_67103,train_70806,train_67103,national,2,1,0,train_70806-train_67103,train_4040,train_71201,[rail],60.0,120.0,[],nan
227597,20220923,P23,5029709,ES243,Zaragoza,44056_AM,ES242,Teruel,NaN,NaN,NF,NF,D05_[50000-inf),D05_[50000-inf),01-02,train,conv_unknown,P23*5029709*5029703*None*train_70806*00-0

In [85]:
#we consider only the trips that have a MMX path, revise this affirmation?
trips_not_commuting_final=trips_not_commuting[trips_not_commuting["path"]!="nan"]
trips_not_commuting_final.loc[:,"mode_tp"]=trips_not_commuting_final["mode_tp"].astype(str)
trips_not_commuting_final_grouped=trips_not_commuting_final.groupby(["origin","destination","path","duration_min","duration_max"], as_index = False)[["trips","archetype_0","archetype_1","archetype_2","archetype_3","archetype_4","archetype_5"]].sum()

In [86]:
trips_not_commuting_final_grouped

,origin,destination,path,duration_min,duration_max,trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5
0,ES111,ES112,"['007122100', '007120300']",240.0,360.0,6.551,5.193586,0.118036,0.059018,0.472144,0.590180,0.118036
1,ES111,ES112,"['007131400', '007120300']",60.0,120.0,12.527,10.376576,0.160420,0.209507,0.764705,0.574008,0.441785
2,ES111,ES112,"['007131400', '007120300']",120.0,180.0,3.735,2.641829,0.022774,0.227744,0.409939,0.204970,0.227744
3,ES111,ES112,"['007131400', '007120300']",240.0,360.0,3.255,2.472010,0.078299,0.044742,0.346753,0.100670,0.212526
4,ES111,ES112,"['007131400', '007120300']",360.0,480.0,4.492,3.602495,0.077832,0.133426,0.378040,0.111188,0.189020
...,...,...,...,...,...,...,...,...,...,...,...,...
12526,ES709,ES708,"['GCXO', 'GCRR']",120.0,180.0,142.247,80.287251,5.251482,4.024842,30.415937,8.901823,13.365665
12527,ES709,ES708,"['GCXO', 'GCRR']",180.0,240.0,140.999,82.928076,4.882438,3.632251,28.192395,10.386914,10.976926
12528,ES709,ES708,"['GCXO', 'GCRR']",240.0,360.0,211.289,114.230318,7.176302,5.533047,47.580615,16.312375,20.456343
12529,ES709,ES708,"['GCXO', 'GCRR']",360.0,480.0,54.408,29.870243,2.314012,0.772183,12.412165,3.985797,5.053602


In [87]:
possible_itineraries_clustered_1_connection=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\processed_baseline\paths_itineraries\possible_itineraries_clustered_0.csv")
possible_itineraries_clustered_2_connections=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\processed_baseline_2_connections\paths_itineraries\possible_itineraries_clustered_0.csv")

In [88]:
possible_itineraries_2_connections["mode_tp"]=possible_itineraries_2_connections["mode_tp"].astype(str)

In [89]:
possible_itineraries_grouped=possible_itineraries_2_connections.groupby(["origin","destination","path","nmodes","mode_tp"],as_index=False)[["access_time","egress_time","travel_time_0","cost_0","emissions_0","mct_time_0_1","travel_time_1","cost_1","emissions_1","mct_time_1_2","travel_time_2","cost_2","emissions_2"]].mean()

In [90]:
possible_itineraries_grouped=possible_itineraries_grouped[possible_itineraries_grouped["nmodes"]!=0]

In [91]:
possible_itineraries_grouped["total_travel_time"]=possible_itineraries_grouped["access_time"].fillna(0)+possible_itineraries_grouped["egress_time"].fillna(0)+possible_itineraries_grouped["travel_time_0"].fillna(0)+possible_itineraries_grouped["travel_time_1"].fillna(0)+possible_itineraries_grouped["travel_time_2"].fillna(0)+possible_itineraries_grouped["mct_time_0_1"].fillna(0)+possible_itineraries_grouped["mct_time_1_2"].fillna(0)
possible_itineraries_grouped["total_cost"]=possible_itineraries_grouped["cost_0"].fillna(0)+possible_itineraries_grouped["cost_1"].fillna(0)+possible_itineraries_grouped["cost_2"].fillna(0)
possible_itineraries_grouped["total_emissions"]=possible_itineraries_grouped["emissions_0"].fillna(0)+possible_itineraries_grouped["emissions_1"].fillna(0)+possible_itineraries_grouped["emissions_2"].fillna(0)

In [92]:
trips_not_commuting_final_grouped=trips_not_commuting_final_grouped.merge(possible_itineraries_grouped, on=["path","origin","destination"],how="left")

In [93]:
trips_not_commuting_final_grouped.head()

,origin,destination,path,duration_min,duration_max,trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,nmodes,mode_tp,access_time,egress_time,travel_time_0,cost_0,emissions_0,mct_time_0_1,travel_time_1,cost_1,emissions_1,mct_time_1_2,travel_time_2,cost_2,emissions_2,total_travel_time,total_cost,total_emissions
0,ES111,ES112,"['007122100', '007120300']",240.0,360.0,6.551,5.193586,0.118036,0.059018,0.472144,0.590180,0.118036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ES111,ES112,"['007131400', '007120300']",60.0,120.0,12.527,10.376576,0.160420,0.209507,0.764705,0.574008,0.441785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ES111,ES112,"['007131400', '007120300']",120.0,180.0,3.735,2.641829,0.022774,0.227744,0.409939,0.204970,0.227744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ES111,ES112,"['007131400', '007120300']",240.0,360.0,3.255,2.472010,0.078299,0.044742,0.346753,0.100670,0.212526,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ES111,ES112,"['007131400', '007120300']",360.0,480.0,4.492,3.602495,0.077832,0.133426,0.378040,0.111188,0.189020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
percentage_of_assigned_od=pd.DataFrame(columns=["origin","destination","num_trips","num_assigned","percent_assigned"])
percentage_of_assigned_od[["origin","destination"]]=trips_not_commuting_final_grouped[["origin","destination"]].drop_duplicates().reset_index(drop=True)

In [101]:
for idx, row in percentage_of_assigned_od.iterrows():
    percentage_of_assigned_od.loc[idx,"num_trips"]=trips_not_commuting[(trips_not_commuting["origin"]==row["origin"])&(trips_not_commuting["destination"]==row["destination"])]["trips"].sum()
    percentage_of_assigned_od.loc[idx,"num_assigned"]=trips_not_commuting_final_grouped[(trips_not_commuting_final_grouped["origin"]==row["origin"])&(trips_not_commuting_final_grouped["destination"]==row["destination"])&(trips_not_commuting_final_grouped["nmodes"].notna())]["trips"].sum()
percentage_of_assigned_od["percent_assigned"]=percentage_of_assigned_od["num_assigned"]/percentage_of_assigned_od["num_trips"]*100
percentage_of_assigned_od[["num_trips","num_assigned","percent_assigned"]]=percentage_of_assigned_od[["num_trips","num_assigned","percent_assigned"]].apply(pd.to_numeric)

In [102]:
percentage_of_assigned_o=pd.DataFrame(columns=["origin","num_trips","num_assigned","percent_assigned"])
percentage_of_assigned_o[["origin"]]=trips_not_commuting_final_grouped[["origin"]].drop_duplicates().reset_index(drop=True)

In [103]:
for idx, row in percentage_of_assigned_o.iterrows():
    percentage_of_assigned_o.loc[idx,"num_trips"]=trips_not_commuting[(trips_not_commuting["origin"]==row["origin"])]["trips"].sum()
    percentage_of_assigned_o.loc[idx,"num_assigned"]=trips_not_commuting_final_grouped[(trips_not_commuting_final_grouped["origin"]==row["origin"])&(trips_not_commuting_final_grouped["nmodes"].notna())]["trips"].sum()
percentage_of_assigned_o["percent_assigned"]=percentage_of_assigned_o["num_assigned"]/percentage_of_assigned_o["num_trips"]*100
percentage_of_assigned_o[["num_trips","num_assigned","percent_assigned"]]=percentage_of_assigned_o[["num_trips","num_assigned","percent_assigned"]].apply(pd.to_numeric)

In [104]:
percentage_of_assigned_o

,origin,num_trips,num_assigned,percent_assigned
0,ES111,13366.753,5981.732,44.750823
1,ES112,1687.156,324.580,19.238292
2,ES113,4022.783,1269.962,31.569240
3,ES114,10447.533,3217.024,30.792188
4,ES120,3420.604,2146.117,62.740879
5,ES130,2953.306,1139.172,38.572772
6,ES211,7671.859,3983.430,51.922617
7,ES212,5798.617,1873.764,32.313981
8,ES213,10022.207,3903.213,38.945643
9,ES220,7055.891,3488.618,49.442629


In [105]:
percentage_of_assigned_o["percent_assigned"].describe()

count     57.000000
mean      49.837648
std       23.672684
min        2.351610
25%       33.860832
50%       46.642208
75%       63.656720
max      100.000000
Name: percent_assigned, dtype: float64

In [106]:
percentage_of_assigned_o.sort_values(by="percent_assigned")

,origin,num_trips,num_assigned,percent_assigned
12,ES242,2359.192,55.479,2.351610
27,ES424,13765.888,911.809,6.623685
11,ES241,2280.247,186.398,8.174465
34,ES514,21311.583,3466.616,16.266347
1,ES112,1687.156,324.580,19.238292
15,ES411,2408.823,469.558,19.493255
32,ES512,14546.059,2921.318,20.083227
10,ES230,2839.704,632.596,22.276829
49,ES620,7558.694,1826.451,24.163579
36,ES522,11295.147,3011.089,26.658254


If one compares this table to the one obtained in new_trips_to_paths1 (without filtering commuter trips), we see that the bad regions are marginally improved but by too little. In my opinion it is more convenient to remove all "Cercanías" trips altogether

In [107]:
station_to_NUTS

{'train_01003': 'ES618',
 'train_01005': 'ES618',
 'train_01007': 'ES618',
 'train_01009': 'ES618',
 'train_02002': 'ES613',
 'train_02003': 'ES617',
 'train_02030': 'ES617',
 'train_03001': 'ES616',
 'train_03100': 'ES616',
 'train_03208': 'ES423',
 'train_03213': 'ES523',
 'train_03216': 'ES523',
 'train_03309': 'ES521',
 'train_03410': 'ES521',
 'train_04007': 'ES424',
 'train_04040': 'ES243',
 'train_04104': 'ES514',
 'train_04307': 'ES512',
 'train_04313': 'ES512',
 'train_05000': 'ES614',
 'train_05012': 'ES614',
 'train_05013': 'ES614',
 'train_05019': 'ES617',
 'train_05101': 'ES111',
 'train_05102': 'ES111',
 'train_05103': 'ES111',
 'train_05104': 'ES111',
 'train_05105': 'ES111',
 'train_05106': 'ES111',
 'train_05107': 'ES111',
 'train_05108': 'ES111',
 'train_05109': 'ES111',
 'train_05111': 'ES111',
 'train_05113': 'ES111',
 'train_05115': 'ES111',
 'train_05117': 'ES111',
 'train_05119': 'ES111',
 'train_05121': 'ES111',
 'train_05123': 'ES111',
 'train_05125': 'ES111',


In [108]:
stops_loc

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,stop_timezone
0,007100367,PORT BOU(fr),42.431666666666665,3.189722222222222,5,nan,Europe/Amsterdam
1,007101003,EL ARAHAL,37.26805555555556,-5.548611111111111,1,nan,Europe/Amsterdam
2,007101005,MARCHENA,37.33416666666667,-5.425555555555556,1,nan,Europe/Amsterdam
3,007101007,OSUNA,37.23388888888889,-5.114999999999999,1,nan,Europe/Amsterdam
4,007101009,PEDRERA,37.2225,-4.893611111111111,1,nan,Europe/Amsterdam
...,...,...,...,...,...,...,...
833,009457000,PORTALEGRE,39.19916666666666,-7.461666666666667,1,nan,Europe/Lisbon
834,009457117,ASSUMAR,39.13861111111111,-7.381944444444444,1,nan,Europe/Lisbon
835,009457174,ARRONCHES,39.096944444444446,-7.333333333333333,1,nan,Europe/Lisbon
836,009457315,SANTA EULALIA-A,39.00611111111111,-7.253333333333333,1,nan,Europe/Lisbon


In [109]:
stops_loc["modified_id"]=stops_loc["stop_id"].apply(lambda x : "train_" + str(x)[4:])

In [110]:
stops_loc

,stop_id,stop_name,stop_lat,stop_lon,location_type,parent_station,stop_timezone,modified_id
0,007100367,PORT BOU(fr),42.431666666666665,3.189722222222222,5,nan,Europe/Amsterdam,train_00367
1,007101003,EL ARAHAL,37.26805555555556,-5.548611111111111,1,nan,Europe/Amsterdam,train_01003
2,007101005,MARCHENA,37.33416666666667,-5.425555555555556,1,nan,Europe/Amsterdam,train_01005
3,007101007,OSUNA,37.23388888888889,-5.114999999999999,1,nan,Europe/Amsterdam,train_01007
4,007101009,PEDRERA,37.2225,-4.893611111111111,1,nan,Europe/Amsterdam,train_01009
...,...,...,...,...,...,...,...,...
833,009457000,PORTALEGRE,39.19916666666666,-7.461666666666667,1,nan,Europe/Lisbon,train_57000
834,009457117,ASSUMAR,39.13861111111111,-7.381944444444444,1,nan,Europe/Lisbon,train_57117
835,009457174,ARRONCHES,39.096944444444446,-7.333333333333333,1,nan,Europe/Lisbon,train_57174
836,009457315,SANTA EULALIA-A,39.00611111111111,-7.253333333333333,1,nan,Europe/Lisbon,train_57315


In [111]:
set(stops_loc["modified_id"])

{'train_00367',
 'train_01003',
 'train_01005',
 'train_01007',
 'train_01009',
 'train_02002',
 'train_02003',
 'train_02006',
 'train_02030',
 'train_03001',
 'train_03100',
 'train_03208',
 'train_03213',
 'train_03216',
 'train_03309',
 'train_03410',
 'train_03412',
 'train_04007',
 'train_04040',
 'train_04104',
 'train_04307',
 'train_05000',
 'train_05012',
 'train_06002',
 'train_06003',
 'train_06006',
 'train_06007',
 'train_07005',
 'train_08004',
 'train_08223',
 'train_08224',
 'train_08240',
 'train_08247',
 'train_08251',
 'train_10000',
 'train_10001',
 'train_10002',
 'train_10005',
 'train_10100',
 'train_10101',
 'train_10103',
 'train_10104',
 'train_10200',
 'train_10201',
 'train_10202',
 'train_10203',
 'train_10204',
 'train_10205',
 'train_10206',
 'train_10207',
 'train_10208',
 'train_10209',
 'train_10300',
 'train_10302',
 'train_10400',
 'train_10409',
 'train_10500',
 'train_10501',
 'train_10502',
 'train_10503',
 'train_10504',
 'train_10600',
 'train_

In [113]:
trips[trips["origin"]=="ES424"]["node_sequence_reduced"].value_counts()

node_sequence_reduced
train_70105-train_70103    402
train_70200-train_70103    392
train_70200-train_70111    183
train_70105-train_70107    157
train_70200-train_18000    147
                          ... 
train_70200-train_17000      1
train_70202-train_18000      1
train_70202-train_70108      1
train_22100-train_08223      1
train_70208-train_70103      1
Name: count, Length: 194, dtype: int64

In [114]:
trips[trips["origin"]=="ES424"]

,date,trip_period,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,distance,route_distance,duration,mode,service,legs,trip_vehicle_type,nationality,home_census,home_zone,overnight_census,income,age,sex,vehicle_type,short_professional_driver,trips,trips_km,sample_trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,duration_min,duration_max,weird_stations
426,20220923,P00,19024,ES424,Guadalajara,2800502,ES300,Madrid,NaN,NaN,NF,H,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*19024*19046*None*train_70105*00-01*road*No...,other,ES,2_28,2800502,2_28,I01_[10000-15000),A02_[45-65),male,passenger,False,3.080,59.878,1.0,2.461449,0.000000,0.063768,0.051014,0.440000,0.063768,3,road-train-road,train_70105-train_70111,train_70105,train_70111,national,2,1,0,train_70105-train_70111,train_4007,train_60000,[rail],60.0,120.0,[]
427,20220923,P00,19046,ES424,Guadalajara,2800501,ES300,Madrid,NaN,NaN,NF,H,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*19046*19046*None*train_70105*00-01*road*No...,other,ES,2_28,2800501,2_28,I01_[10000-15000),A00_[0-25),female,passenger,False,9.128,126.992,1.0,7.709459,0.000000,0.000000,0.370054,0.801784,0.246703,3,road-train-road,train_70105-train_70103,train_70105,train_70103,national,2,1,0,train_70105-train_70103,train_4007,train_60000,[rail],60.0,120.0,[]
428,20220923,P00,19046,ES424,Guadalajara,2800501,ES300,Madrid,NaN,NaN,O,O,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*19046*19046*None*train_70105*00-01*road*No...,other,ES,2_28,28053,2_28,I01_[10000-15000),A02_[45-65),male,passenger,False,2.398,30.986,1.0,1.916414,0.000000,0.049648,0.039718,0.342571,0.049648,3,road-train-road,train_70105-train_70103,train_70105,train_70103,national,2,1,0,train_70105-train_70103,train_4007,train_18000,[rail],0.0,60.0,[]
429,20220923,P00,19046,ES424,Guadalajara,2800505,ES300,Madrid,NaN,NaN,NF,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*19046*19046*None*train_70105*00-01*road*No...,other,ES,2_28,2800505,2_28,I01_[10000-15000),A03_[65-100),female,passenger,False,6.485,99.284,1.0,5.215210,0.000000,0.136049,0.226748,0.680245,0.226748,3,road-train-road,train_70105-train_70103,train_70105,train_70103,national,2,1,0,train_70105-train_70103,train_4007,train_18000,[rail],0.0,60.0,[]
430,20220923,P00,19046,ES424,Guadalajara,2800505,ES300,Madrid,NaN,NaN,NF,NF,D04_[10000-50000),D04_[10000-50000),03-04,train,conv_unknown,P00*19046*19046*None*train_70105*00-01*road*No...,other,FR,NaN,NaN,NaN,NaN,NaN,NaN,passenger,False,1.599,23.276,1.0,1.287506,0.000000,0.036341,0.025958,0.244003,0.005192,3,road-train-road,train_70105-train_70103,train_70105,train_70103,national,2,1,0,train_70105-train_70103,train_70200,train_10200,[rail],180.0,240.0,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226858,20220923,P23,1913005,ES424,Guadalajara,2814803,ES300,Madrid,NaN,NaN,NF,H,D04_[10000-50000),D04_[10000-50000),02-03,train,conv_unknown,P23*1913005*1913005*None*train_70200*00-01*roa...,other,ES,2_28,2814803,2_28,I01_[10000-15000),A00_[0-25),male,passenger,False,10.487,391.953,1.0,8.959767,0.000000,0.101816,0.407262,0.712709,0.305447,3,road-train-road,train_70200-train_70102,train_70200,train_70102,national,2,1,0,train_70200-train_70102,train_70200,train_17000,[rail],120.0,180.0,[]
226859,20220923,P23,19171,ES424,Guadalajara,2800501,ES300,Madrid,NaN,NaN,NF,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P23*19171*1913005*None*train_70200*00-01*road*...,other,ES,2_28,2800501,2_28,I01_[10000-15000),A03_[65-100),female,passenger,False,4.81

stations that I want to remove:
- In the province of Guadalajara:
    - Guadalajara: train_70200
    - Azuquecas: train_70105
- In the province of Madrid:
    - Alcalá de Henares Universidad: train_70107
    - Alcalá de Henares: train_70103
    - Torrejón de Ardóz: train_70102
    - La Garena: train_70111

In [ ]:
# Lists of cercanias stations that screw the assignation
stations_madrid = ['train_70102', 'train_70103',"train_70107","train_70111"] # corresponding to torrejon de ardoz, alcala de henares, the university of alcala, and la garena
stations_guadalajara = ['train_70105', 'train_70200'] # corresponding to azuquecas and guadalajara

# Define function to check if a row should be eliminated
def eliminate_cercanias(node_sequence):
    # Split the string into individual stations
    stations = node_sequence.split('-')
    start, end = stations[0], stations[-1]  # Get the first and last station
    
    # Check elimination conditions
    return ((start in stations_madrid and end in stations_guadalajara) or
            (start in stations_guadalajara and end in stations_madrid))

In [127]:
# Apply the function to filter out rows
tripsguada = trips[~trips['node_sequence_reduced'].apply(eliminate_cercanias)]

In [128]:
tripsguada[tripsguada["origin"]=="ES424"]["node_sequence_reduced"].value_counts()

node_sequence_reduced
train_70200-train_18000    147
train_70105-train_17000     97
train_70105-train_18000     90
train_70200-train_70112     80
train_17000                 66
                          ... 
train_70200-train_17000      1
train_70202-train_18000      1
train_70202-train_70108      1
train_22100-train_08223      1
train_70208-train_70103      1
Name: count, Length: 186, dtype: int64

In [124]:
tripsguada=trips[~(((trips["node_sequence_reduced"].str.startswith("train_70200"))|(trips["node_sequence_reduced"].str.startswith("train_70105")))&((trips["node_sequence_reduced"].str.endswith("train_70103"))|(trips["node_sequence_reduced"].str.endswith("train_70107"))|(trips["node_sequence_reduced"].str.endswith("train_70111"))))]

In [125]:
tripsguada[tripsguada["origin"]=="ES424"]["node_sequence_reduced"].value_counts()

node_sequence_reduced
train_70200-train_18000    147
train_70200-train_70102    103
train_70105-train_17000     97
train_70105-train_70102     91
train_70105-train_18000     90
                          ... 
train_70200-train_17000      1
train_70202-train_18000      1
train_70202-train_70108      1
train_22100-train_08223      1
train_70208-train_70103      1
Name: count, Length: 188, dtype: int64

In [116]:
trips["start_node_reduced"].str.startswith("train")

0         True
1         True
3         True
4         True
5         True
          ... 
227596    True
227597    True
227600    True
227601    True
227602    True
Name: start_node_reduced, Length: 129502, dtype: bool